In [41]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import glob
# from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from itertools import chain #untuk flaten array 2d
from collections import defaultdict
from collections import OrderedDict


In [42]:
def readFile(directory):
    listFile = glob.glob(directory+"/*.txt")
    return listFile

## normalisasi

In [43]:
def normalisasi(body):
    
    hasil = []
    temp_str = ''

    #hapus tag
    hapusTag = r'<DATE>|</DATE>|<TITLE>|</TITLE>|<BODY>|</BODY>'
    body = re.sub(hapusTag, ' ', body)
    
    #hapus \n
    hapusEnter = r'\n'
    body = re.sub(hapusEnter, ' ', body)
    
    #hapus spasi lebih dari satu
    spasiLebihDariSatu = r'\s+'
    body = re.sub(spasiLebihDariSatu, ' ', body)
    
    # hapus htmlEntities 
    htmlEntities = r"&lt;"
    body = re.sub(htmlEntities, '', body)
    
    #ambil tanggal
    tanggal = r"((\d{2}|\d{4})[.-](J(anuary|an|AN|une|un|UN|uly|ul|UL)|F(ebruary|eb|EB)|M(arch|ar|AR|ay|AY)|A(pril|pr|PR|ugust|ug|UG)|S(eptember|ep|EP)|O(ctober|ct|CT)|N(ovember|ov|OV)|D(ecember|ec|EC))[-.](\d{4}|\d{2}))"
    list_tanggal = re.findall(tanggal,body)
    hasil.append(list_tanggal)
    body = re.sub(tanggal, "", body)
    
    #ambil waktu
    waktu = r"(\d{2}:\d{2}:\d{2}(\.)?\d{2})"
    list_waktu = re.findall(waktu,body)
    hasil.append(list_waktu)
    body = re.sub(waktu, "", body)
    
    #ambil angka desimal, persen, mata uang 
    desimal = "(([+-]?(\$|£)?(\d{1,3}[.,])*(\d+)(%)?))"
    list_desimal = re.findall(desimal, body)
    hasil.append(list_tanggal)
    body = re.sub(desimal, "", body)
    
    #email
    email = "([\w-]+(\.[\w-]+|\.)*@[\w-]+(\.[\w-]+)+)"
    list_email = re.findall(email,body)
    hasil.append(list_email)
    body = re.sub(email, "", body)
    
    #url
    url = "((http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/.*)?)"
    list_url = re.findall(url,body)
    hasil.append(list_url)
    body = re.sub(url, "", body)
    
    tanda_baca = r'["()<>+?.\[\]{}:,\'\'/-]'
    body = re.sub(tanda_baca, ' ', body)

    body = re.sub(spasiLebihDariSatu, ' ', body)
    
    kutips = r"'s"
    body = re.sub(kutips, '', body)
    
    list_flat = list(chain.from_iterable(hasil))
    
    for i in list_flat:
        temp_str = temp_str + i[0] + " "
    
    body = body + " " + temp_str
    
    body = re.sub(spasiLebihDariSatu, ' ', body)
    
    return body

In [44]:
def readText(file):
    words = []
    f = open(file, 'r') #open file
    text = f.read()    
    f.close()

    return text

In [45]:
def removeEnter(listOfWords): #['<BODY>\n']
    for i in range(len(listOfWords)):
        listOfWords[i] = listOfWords[i].replace('\n','')
        
    return listOfWords

In [46]:
def removeNull(listOfWords): #['','','eat','food','','']
    listOfWords = list(filter(None, listOfWords))
    
    return listOfWords

In [47]:
def caseFolding(listOfWords):
    for i in range(len(listOfWords)):
        listOfWords[i] = listOfWords[i].casefold()
        
    return listOfWords

<h3>Load Stop Word from NLTK</h3>

In [48]:
stop_words = stopwords.words('english')

In [49]:
def removeStopWord(listOfWords):
    for i in listOfWords:
        if i in stop_words:
            listOfWords.remove(i)
        
    return listOfWords

<h3>Load Lemmatizer from NLTK</h3>

In [50]:
lemmatizer = WordNetLemmatizer()

In [51]:
def lemmatization(listOfWords):
    for i in range(len(listOfWords)):
        listOfWords[i] = lemmatizer.lemmatize(listOfWords[i])
        
    return listOfWords

<h3>Load Stemming from NLTK</h3>

In [52]:
def stemming(listOfWords):
    stemmer = SnowballStemmer("english")
    stemmed = [stemmer.stem(word) for word in listOfWords]
    
    return stemmed

# Token

In [53]:
def make_token(txt):
    list_hasil = txt.split(" ")            
    return list_hasil

# Term

In [54]:
def make_term(file_name_token,file_name_term):
    list_term = []
    f = open(file_name_token, 'r')
    text = f.read()
    f.close()
    
    list_token = text.split("\n")
    for i in list_token:
        if i not in list_term:
            list_term.append(i)
            
    f1 = open(file_name_term,'a+')
    text = concate_term_newline(list_term)
#     print(text)    
    f1.write(text)
    f1.close()


In [55]:
def concate_term_newline(dist_in):
    hasil = ''
    str_temp2 = ''
    is_first = True
    for i in dist_in:
        for j in dist_in[i]:
            if is_first:
                str_temp2 = j
                is_first = False
            else:
                str_temp2 = str_temp2 + ","+ j
        is_first = True
        str_temp = i+ "\t" + str_temp2+ "\n"
        str_temp2 = ""
        hasil = hasil + str_temp
        
    return hasil

# tulis data

In [56]:
def writeFile(dict_in,file_name):
    
    str_of_list = concate_term_newline(dict_in)
    
#     print(str_of_list)
    saveFile = open(file_name, 'w+' )
    saveFile.write(str_of_list)
    saveFile.close()
#     print(list_hasil)
    

<h2>Add Doc Number</h2>

In [57]:
def addDocNumber(_listIn, _docNum):
#     _docNum[14:18]
    for i in range(len(_listIn)):
        _listIn[i] = _listIn[i] + '_' + _docNum[14:18]
        
    return _listIn

<h1>Main</h1>

In [58]:
listOfFiles = readFile('DataRouter')
listOfWordsOfFile = []
counter = 1
blok = 10

dict_list_term = defaultdict(list)
blok_file_name = "term_blok"
blok_name = []

for i in listOfFiles: #iterasi tiap file
    txt = readText(i)
    
    txt = normalisasi(txt)
    listOfWords = make_token(txt)
    listOfWords = removeNull(listOfWords)
    listOfWords = caseFolding(listOfWords)
    listOfWords = removeStopWord(listOfWords)
    listOfWords = stemming(listOfWords)
    listOfWords = lemmatization(listOfWords)
    
#     Diganti KEVIN
#     listOfWords = addDocNumber(listOfWords,i)
    
    
    for j in listOfWords:
#         if j not in list_term:
#         list_term[j].append(i) #15:01:01.79 : ,DataRouter\Doc0001.txt
        dict_list_term[j].append(i[14:18]) #15:01:01.79 : ,0001
    
#sorting ascending
dict_list_term = OrderedDict(sorted(dict_list_term.items()))

writeFile(dict_list_term,"hasil_inverted.txt")
#     counter = counter + 1
#     print(counter)
#     print(list_term)


In [59]:
# for i in dict_list_term:
#     print(i, ":", list_term[i])